<div id="header">
        <p style="float: left; width: 34%; text-align: left;">VIDHI JATIN SHAH</p>
    <h1 style="float: left; width: 33%; text-align: center;">LAB 2-Activity 2</h1>
    <p style="float: left; width: 33%; text-align: right;">UBPerson No:50207090</p>
</div>

1. Imports all the important libraries
  * RSQLite<-Working with SQL database
  * dplyr<-Working with SQL database
  * sqldf<-to use sql statements on dataframe 
  * plyr<-Use for combining two dataframes

In [18]:
library("RSQLite")
library("dplyr")
library("sqldf")
library(plyr)

* Connect to sqlite db file

In [19]:
db <- dbConnect(SQLite(), dbname="database.sqlite")

* List all the tables in the Databse

In [20]:
dbListTables(db) 

[1] "Country"           "League"            "Match"            
[4] "Player"            "Player_Attributes" "Team"             
[7] "Team_Attributes"   "sqlite_sequence"

* List all the columns in a table

In [21]:
dbListFields(conn=db,"League")

[1] "id"         "country_id" "name"

* dbGetQuery is used to fire query on sql db 
* Two tables Player and Player_attributes are join on the column player_api_id
* We sum all the gk attributes<- gk_diving+gk_handling+gk_kicking+gk_positioning+gk_reflexes
* Each player is assign more than one overall rating so we take the max overall_rating out of all

In [22]:
p1<-dbGetQuery(conn=db,"select p.player_api_id,p.player_name,
max(pa.overall_rating) as overall_rating, max(gk_diving+gk_handling+gk_kicking+gk_positioning+gk_reflexes) as gk 
from Player as p inner join Player_Attributes pa on p.player_api_id=pa.player_api_id
group by p.player_api_id,player_name")

* We count the distinct League Ids of all the players and store it in a new column league_count
* To find the number of appearances of each player we count each player_api_id for each of the match he has played in.

In [23]:
p2<-dbGetQuery(conn=db,"select p.player_api_id,count(player_api_id) as appearances,count(Distinct league_id) as league_count from Player as p,Match as m where 
p.player_api_id=m.home_player_1 OR 
p.player_api_id=m.home_player_2 OR
p.player_api_id=m.home_player_3 OR
p.player_api_id=m.home_player_4 OR
p.player_api_id=m.home_player_5 OR
p.player_api_id=m.home_player_6 OR
p.player_api_id=m.home_player_7 OR
p.player_api_id=m.home_player_8 OR
p.player_api_id=m.home_player_9 OR 
p.player_api_id=m.home_player_10 OR 
p.player_api_id=m.home_player_11 OR 
p.player_api_id=m.away_player_1 OR 
p.player_api_id=m.away_player_2 OR 
p.player_api_id=m.away_player_3 OR 
p.player_api_id=m.away_player_4 OR 
p.player_api_id=m.away_player_5 OR 
p.player_api_id=m.away_player_6 OR 
p.player_api_id=m.away_player_7 OR 
p.player_api_id=m.away_player_8 OR 
p.player_api_id=m.away_player_9 OR 
p.player_api_id=m.away_player_10 OR 
p.player_api_id=m.away_player_11 group by p.player_api_id,p.player_name ")

* Merge two tables p1 and p2 on player_api_id which stores all the necessary information of Players such as player_api_id,gk,overall_rating,player_name,league_count,appearances.

In [24]:
FinalPlayer<-merge(p1,p2,by=c("player_api_id","player_api_id"))

* Write the Players dataframe to csv

In [25]:
write.csv(FinalPlayer,file="Player.csv",row.names=FALSE)

* coalesce is used to replace all NA values in the column with a specified value i.e 0 in our case.
* We sum the buildUpPlay attributes,ChanceCreation Attributes and Defence attributes of all the players.
* We perform inner join on two tables team and team_Attributes on the column team_api_id

In [26]:
t1<-dbGetQuery(conn=db,"Select t.team_api_id,t.team_short_name ,
max((coalesce(buildUpPlayDribbling,0)+coalesce(buildUpPlaySpeed,0)+coalesce(buildUpPlayPassing,0)))as buildUpPlay,
max(chanceCreationPassing+chanceCreationCrossing+chanceCreationShooting)as chanceCreation,
max(defencePressure+defenceAggression+defenceTeamWidth)as defence
from  team t inner join team_Attributes ta on t.team_api_id=ta.team_api_id group by t.team_api_id,t.team_short_name")

* Sum the number of goals scored by each team from home_team_goals and away_team_goal columns

In [27]:
x1<-dbGetQuery(conn=db,"Select home_team_api_id as team_api_id, sum(home_team_goal) as goals from Match group by home_team_api_id")
x2<-dbGetQuery(conn=db,"Select away_team_api_id as team_api_id , sum(away_team_goal) as goals from Match group by away_team_api_id")

* combines two dataframes that has goal count for home_teams and away_teams and finally merges on a common column team_api_id to give total number of goals for each team.

In [28]:
x4<-ddply(rbind(x1, x2), .(team_api_id), summarize, number_of_goals = sum(goals))

* Gives a final dataframe that has all the necessary details needed for a Team.

In [29]:
x5<-merge(x=t1,y=x4,by.x='team_api_id',by.y='team_api_id')

* Write the teams dataframe to csv

In [30]:
write.csv(x5,file="Team.csv",row.names=FALSE)